# Temporal and spatial mismatch vs $t_0$: (2,2) mode only

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import qnmfits
import CCE
from qnmfits.spatial_mapping_functions import *

plt.style.use("stylesheet.mplstyle")
fig_width = 246.0 * (1.0 / 72.27)
fig_height = fig_width / 1.618

In [2]:
id = "0001"

sim_main = CCE.SXS_CCE(id, lev="Lev5", radius="R2")
sim_options = [("Lev4", "R2"), ("Lev4", "R3"), ("Lev5", "R3")]

sims = []

for lev, R in sim_options:
    try:
        sim = CCE.SXS_CCE(id, lev=lev, radius=R)
        sims.append(sim)
    except:
        pass

In [3]:
l_max = 8
n_max = 7

In [4]:
t0_array = np.linspace(-30, 100, 100)
mode_list = [[(2, 2, n, 1) for n in range(N + 1)] for N in range(1, n_max + 1)]

In [5]:
mm_array = np.zeros((len(mode_list), len(t0_array)))

for i, modes in enumerate(mode_list):

    mm_list = qnmfits.mismatch_t0_array(
        sim_main.times,
        sim_main.h[2, 2],
        modes,
        Mf=sim_main.Mf,
        chif=sim_main.chif_mag,
        t0_array=t0_array,
    )

    mm_array[i] = mm_list

In [6]:
for i, sim in enumerate(sims):
    shifts = np.arange(-0.1, 0.1, 0.0001)
    shift_idx = np.argmin([data_mismatch(sim_main, sim, modes=[(2, 2)], shift=s) for s in shifts])
    sims[i].zero_time = -shifts[shift_idx]
    sims[i].time_shift()

hsm_list = np.zeros((len(t0_array)))
for i, t0 in enumerate(t0_array):
    for sim in sims:
        hsm = data_mismatch(sim_main, sim, modes=[(2, 2)], t0=t0)
        if hsm > hsm_list[i]:
            hsm_list[i] = hsm

/tmp/ipykernel_1962484/2955941574.py:2: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors1 = plt.cm.get_cmap("viridis", n_max + 1)


In [7]:
fig, ax = plt.subplots(figsize=(fig_width, fig_height))
colors1 = plt.cm.get_cmap("viridis", n_max + 1)

for i, modes in enumerate(mode_list):

    line_error_index = np.less(hsm_list, mm_array[i])
    first_false_index = np.argmax(~line_error_index)

    if not np.all(line_error_index):
        line_error_index[first_false_index:] = False

    ax.plot(
        t0_array[line_error_index],
        mm_array[i][line_error_index],
        label=f"${modes[-1][-2]}$",
        color=colors1(i),
    )
    ax.plot(
        t0_array[first_false_index - 1 : first_false_index + 1],
        mm_array[i][first_false_index - 1 : first_false_index + 1],
        color=colors1(i),
        linestyle="--",
    )
    ax.plot(
        t0_array[~line_error_index],
        mm_array[i][~line_error_index],
        color=colors1(i),
        linestyle="--",
    )

ax.fill_between(t0_array, hsm_list, color="grey", alpha=0.3)

ax.set_yscale("log")
ax.set_xlim(t0_array[0], t0_array[-1])
ax.set_xlabel("$t_0 \ [M]$")
ax.set_ylabel("$\mathcal{M}^{22}$")

_ = ax.legend(
    ncol=2,
    frameon=False,
    loc="upper right",
    title=r"$n_{\mathrm{max}}$",
    labelspacing=0.1,
)

fig.savefig("figs/mismatch_t0_22mode.pdf", bbox_inches="tight")